## Evaluate Trained Robot Policy Model and Visualize

In [1]:
import os
import sys

import torch

In [2]:
# get the current script's directory
current_directory = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
# get the parent directory
parent_directory = os.path.dirname(current_directory)
# add the parent directory to the sys.path
sys.path.append(parent_directory)

In [3]:
from utils import constants
from utils.config import Config
from utils.dataset_loader import PolicyDatasetLoader

from optimization.functions import setup_config, get_directories, load_policy

from models.policy_model import RobotPolicy

In [4]:
# available evaluating machine
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Evaluating Device: ", device)

# setup hyperparameters
configs = setup_config(device=device)

# create and return preliminary base paths
json_paths, results_path = get_directories(parent_directory=parent_directory)

Evaluating Device:  cpu
Current Time:  Jan_29_2024-19_50_56


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Evaluating Device: ", device)
configs.device = device

Evaluating Device:  cpu


In [6]:
policy_network = RobotPolicy(state_size=configs.state_size,
                             hidden_size=configs.hidden_size,
                             out_size=configs.action_size,
                             log_std_min=configs.policy_log_std_min,
                             log_std_max=configs.policy_log_std_max,
                             log_std_init=configs.policy_log_std_init,
                             device=configs.device)

In [7]:
# load demonstrations dataset
training_data = PolicyDatasetLoader(demo_data_json_paths=json_paths)


================== Policy Dataset Loader ==================

Number of Trajectories:  43
Each Trajectory Length:  20
Full Demo Dataset Size:  922


# Functions

# Test

In [8]:
# folder name where parameters are located ("results / policy_network_params / loading_folder_name")
loading_folder_name = "Jan_29_2024-18_35_10"

In [9]:
params_name = "policy_network_epoch_100_loss_3_19916.pt"

In [10]:
# location of the trained model parameters (make sure that the folder exists where model is trained priorly)
model_folder_path = os.path.join(results_path, "policy_network_params", loading_folder_name)

In [11]:
model_path = os.path.join(model_folder_path, params_name)

In [12]:
policy_network = load_policy(policy_network=policy_network,
                             model_path=model_path)

In [13]:
training_data[25]

(tensor([0.2958, 0.7073, 0.2999, 0.5409]),
 tensor([-0.3162,  0.0008,  0.4145]),
 1,
 5)

In [14]:
training_data[12]

(tensor([0.4475, 0.4495, 0.7963, 0.4696]),
 tensor([-0.2571,  0.4895,  0.3877]),
 0,
 12)